In [90]:
import numpy as np
import pandas as pd
from sklearn import linear_model

In [91]:
def dataclean(x):
    for i in range(x.shape[1]-2):
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('#','')
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('x','')
        x.loc[:,str(i)] = x.loc[:,str(i)].str.replace('*','')
    x = x.replace({'NR':0})
    columns = []
    for i in range(x.shape[1]-2):
        columns.append(str(i))
    x[columns] = x[columns].astype(np.float)
    x = x.fillna(0)
    return x

In [92]:
def splitdata(x):
    y = x[x['測項'] == 'PM2.5'].iloc[:,2:]
    x = x.drop(['日期','測項'] , axis=1)
    y = y.to_numpy()
    x = x.to_numpy()
    newdata = x[0:18,:]
    for i in range(int(x.shape[0]/18-1)):
        newdata = np.hstack((newdata,x[18*(i+1):18*(i+2),:]))    
    y = y.reshape(-1,1)
    train_X = []
    for i in range(newdata.shape[1]-9):
        train_X.append(newdata[:,i:i+9])
    train_X = np.array(train_X)
    y = y[9:]
    return train_X,y

In [93]:
def valid(x, y):
    if y <= 2 or y > 100:
        return False
    if ((x <= 2).sum() + (x > 100).sum()) > 0:
        return False
    return True
def datafilter(x,y):
    xx = []
    yy = []
    for i in range(x.shape[0]):
        if valid(x[i][9],y[i]):
            xx.append(x[i].reshape(1,-1))
            yy.append(y[i])
    train_X = np.array(xx)
    train_Y = np.array(yy)
    train_Y = train_Y.reshape(-1,1)
    train_X = train_X.reshape(-1,162)        
    return train_X,train_Y       

In [94]:
def lse(x,y):
    reg = linear_model.Ridge(alpha=0.01)
    reg.fit(x,y)
    return reg.coef_

In [95]:
x1 = pd.read_csv('year1-data.csv')
x2 = pd.read_csv('year2-data.csv')
data = pd.concat([x1,x2])
data = dataclean(data)
x, y = splitdata(data)
x, y = datafilter(x,y)
w = lse(x,y)
w = w.reshape(-1,1)

In [17]:
def rmse(x,y):
    mse = 0
    for i in range(len(y)):
        mse += (y[i]-x[i])**2
        mse = np.sqrt(mse/len(y))
    return mse

In [78]:
rmse(np.dot(x,w),y)

array([0.03668108])

In [96]:
test = pd.read_csv('testing_data.csv')
test = dataclean(test)
test = test.drop(['id','測項'],axis=1)
test = test.to_numpy()
test = test.reshape(-1,162)

In [97]:
predict = pd.DataFrame(columns = ['id','value'])
ids = []
values = []
for i in range(500):
    ids.append('id_'+ str(i))
    values.append(float(np.dot(test,w)[i]))
predict['id'] = ids
predict['value'] = values  
predict.to_csv('HW1_predict_best.csv' , index = False)

In [98]:
w_best = pd.DataFrame(w)
w_best.to_csv('w_best.csv',index=False)